In [190]:
"""
Testing strategies
"""
import random
import statistics
import heapq


def get_random(hand, other, window=12):
    # Select randomly from possible values 
    guess = random.sample(list(other), window)

    # Get number of correct guesses  
    num_correct = len(set(hand).intersection(set(guess)))
    return guess, num_correct


def get_below_max(hand, other, window=12):
    # Max strategy 
    max_val = max(hand)
    below = [val for val in other if val < max_val]

    # Select randomly from possible values 
    guess = random.sample(below, window)

    # Get number of correct guesses  
    num_correct = len(set(hand).intersection(set(guess)))
    return guess, num_correct


def get_most_in_window(hand, other, window=12, highest=52):
    # Just for easier debugging
    hand.sort()
    
    # In YOUR HAND finding the window with the highest number of cards 
    min_window = 0
    num_cards_in_window = 0
    for min_card in hand:
        # Values in window that are ABOVE the min_window but BELOW the highest value (ie 52) 
        vals_in_window = [i for i in range(min_card + 1, min_card + window + 1) if i <= highest]
        
        # This means that min_window > 40 (because then the numbers to check for go above 53 which is out of range).
        if len(vals_in_window) < window:
            looped_window = [i for i in range(1, (min_card + window + 1) - highest)]
            vals_in_window = vals_in_window + looped_window
        
        cards_in_window = set(hand).intersection(set(vals_in_window))
        
        if len(cards_in_window) > num_cards_in_window:
            num_cards_in_window = len(cards_in_window)
            min_window = min_card
            
    # Your PARTNER using that min_window to get their guesses
    # Now use this min_window to guess cards from other 
    guess = [val for val in other if min_window < val]
    
    # If there are not enough guessable cards above the min_window loop back to select from the bottom
    if len(guess) < window:
        max_window = window - len(guess) 
        other.sort()
        guess = guess + other[:max_window]
    else:
        guess = guess[:window]
        
    assert len(guess) == window
    
    # Get number of correct guesses  
    num_correct = len(set(hand).intersection(set(guess)))
    return guess, num_correct

def get_gap_score(hand, other, window=12):
    
    gap = get_largest_gap(hand)
    max_gap = gap[0]
    
    # Guess values above the maximum of the calculated gap 
    above = [val for val in other if max_gap < val]
    above.sort()
    if len(above) < window:
        num_looped = window - len(above) 
        other.sort()
        guess = above + other[:num_looped]
    else: 
        guess = above[:window]
    
    # Get number of correct guesses  
    num_correct = len(set(hand).intersection(set(guess)))
    return guess, num_correct
    
    
    
def get_largest_gap(cards: list[int]) -> list[(int,int)]:
    """
    :param cards: list of integers representing the player's hand

    return an array of length NUM_GAP_ROUNDS / 2 conveying the largest gaps in our hand
    Each value in the array is a tuple (x,y) where x is the value larger than the values in the gap,
    and y is the value smaller than the values in the gap.
    Note that this is cyclic, so if x < y, then we know that there are no values less than x and
    no values greater than y.
    """
    num_gaps = 1
    if num_gaps < 1: return []

    gap_dict = {}
    cards = cards.copy()
    cards.sort()

    for i in range(len(cards)-1):
        card = cards[i]
        next_card = cards[(i+1)%len(cards)]
        gap = (next_card-card)%51
        if i < len(cards)-1:
            # special considerations for "cycle" piece
            gap -= 1

        if gap > 0:
            # if these are not in a row, then there is a gap
            gap_dict[(next_card,card)] = gap

    return heapq.nlargest(num_gaps, gap_dict, key=gap_dict.get)[0]




In [199]:
random_score = []
max_score = []
gaps_score = []
window_score = []

gap_sizes = []

num_runs = 10000
for exp in range(0,num_runs):
    deck = [x for x in range(1,53)]
    hand = random.sample(deck, 13)
    part = random.sample(list(set(deck) - set(hand)), 13)
    other = list(set(deck) - set(part))
    
    r_guess, r_num_correct = get_random(hand, other)
    random_score.append(r_num_correct)
    m_guess, m_num_correct = get_below_max(hand, other)
    max_score.append(m_num_correct)
    w_guess, w_num_correct = get_most_in_window(hand, other)
    window_score.append(w_num_correct)
    g_guess, g_num_correct = get_gap_score(hand, other)
    gaps_score.append(g_num_correct)
    
    # calculate largest gaps 
    gap = get_largest_gap(hand)
    if gap[0] > gap[1]:
        size = gap[0] - gap[1]
    else:
        print("Looped gap: ", gap)
        size = (52 - gap[0]) + gap[1]
    gap_sizes = gap_sizes + [size]

print(f"------- Round 1 Scores Over {num_runs} runs -------")
print("Random selection -- Avg: ", statistics.mean(random_score), "SD: ", round(statistics.stdev(random_score), 3))
print("Max bound selection -- Avg: ", statistics.mean(max_score), "SD: ", round(statistics.stdev(max_score), 3))
print("Gap with max gap selection -- Avg: ", statistics.mean(gaps_score), "SD: ", round(statistics.stdev(gaps_score), 3))
print("Most cards in window selection -- Avg:", statistics.mean(window_score), "SD: ", round(statistics.stdev(window_score), 3))
print(" ")
print("------- Other ------- ")
print(f"Gaps: Avg size: {statistics.mean(gap_sizes)}, SD: {statistics.stdev(gap_sizes)}")


------- Round 1 Scores Over 10000 runs -------
Random selection -- Avg:  4.0054 SD:  1.381
Max bound selection -- Avg:  3.9944 SD:  1.391
Gap with max gap selection -- Avg:  4.3182 SD:  1.291
Most cards in window selection -- Avg: 5.3876 SD:  0.953
 
------- Other ------- 
Gaps: Avg size: 10.5631, SD: 2.947318621850354
